In [11]:
from transformers import AutoTokenizer, AutoModel, pipeline
import torch
import numpy as np
import pandas as pd
import pickle

In [2]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")

In [10]:
DATA_PATH_IN = '../data/'
DATA_PATH_OUT = '../data/Experiment_no_Class/'

In [12]:
with open(DATA_PATH_IN + 'text_dataset.pkl', 'rb') as f:
    text_data = pickle.load(f)

*Credits: https://towardsdatascience.com/how-to-do-average-and-max-word-embedding-for-long-sentences-f3531e99d998*

In [13]:
def chunking(max_len, sent):
    """because the embedding function is trained on dim 512, so we have to limit the size of the sentences using max_len so the final chunked sentences wont exceed length 510
    Args:
        max_len (int): maximum number of tokens for each chunk
        sent (str): input sentence
    Returns:
        sent_chunks (List(str)): list of chunked sentences
    """

    bert_tokenized_text = tokenizer.tokenize(sent)


    if len(bert_tokenized_text) > max_len:
        # using list comprehension to divide the sequence
        final = [
            bert_tokenized_text[i * max_len : (i + 1) * max_len]
            for i in range((len(bert_tokenized_text) + max_len - 1) // max_len)
        ]

        # join back to sentences for each of the chunks
        sent_chunks = []
        for item in final:
            try:
                # make sure the len(items) > 1 or else some of the embeddings will appear as len 1 instead of 768.
                assert len(item) > 1
            except Exception as e:
                print(item, e)
            sent_chunks.append(" ".join(item))
        return sent_chunks

    else : return sent

In [9]:
len(chunking(5, """def embed_single_row_chunking(chunk): tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
    row_embeddings = model(torch.tensor(tokens_ids)[None,:])[0]"""))

14

In [14]:
def embed_single_row_chunking(chunk):

    '''
    gets a tokenized text chunk (code or md) and returns embeddings tensor
    
    '''

    tokens = [tokenizer.cls_token] + chunk + [tokenizer.sep_token]

    tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
    row_embeddings = model(torch.tensor(tokens_ids)[None,:])[0]

    if row_embeddings is not None:
        return row_embeddings


In [15]:
def average_embeddings_on_chunks(embedding_tensor):

    '''
    gets the returned embeddings tensor of a sequence and averages it 
    to return one 768 dim vector for each row in the dataframe

    '''

    avg_rep = np.empty(1)
    if embedding_tensor is not None:
        if type(embedding_tensor) == torch.Tensor:
            # convert tensor into np array
            tensor_np = embedding_tensor.cpu().detach().numpy()
            # average of embeddings of the tokens in the sequence
            avg_rep = np.mean(tensor_np[0], axis=0)
        else: 
            tensor_np = embedding_tensor
            # average of embeddings of the tokens in the sequence
            avg_rep = np.mean(tensor_np, axis=0)
        # return average representation of a sequence of tokens
        return avg_rep

In [ ]:
row_avg_embedding = []
for row in text_data.index[94454:188908]:
    embedding = embed_single_row_chunking(text_data.loc[row, 'source'], row)
    row_avg_embedding.append((average_embeddings_on_chunks(embedding),  text_data.loc[row, 'title'], text_data.loc[row, 'tag']))
    print(row)
# build intermediary dataframe of averaged row vectors, titles and tags 
interm_df02 = pd.DataFrame(row_avg_embedding, columns = ['row_vector', 'title', 'tag'])